In [ ]:
import holoviews as hv
import numpy as np
import pandas as pd
hv.extension(case_sensitive_completion=True)
hv.extension('bokeh')
from readout_ep_output import walk_ep_dirs_old, extract_data_from_ep_dirs

In [ ]:
directory = r'X:\David\THY_1_GCaMP_BEFOREAFTER_TAC_290517\results'
list_of_wanted = ['C_df']
df = extract_data_from_ep_dirs(directory=directory, file_end=r"*compiled.mat",
                                   list_of_wanted=list_of_wanted)
df.head()

In [ ]:
%%opts Curve [width=500 height=400 tools=['hover']]

ids = np.unique(df.index.get_level_values('animalID'))
conds = np.unique(df.index.get_level_values('conditionID'))
days = np.unique(df.index.get_level_values('daysAfterBaseline'))

gridspace = hv.GridSpace(kdims=['Day', 'Condition'], group='Individual Mice', label='Calcium Traces')

for day in days:
    for cond in conds:
        holomap = hv.HoloMap(kdims=['Mouse ID'])
        for mid in ids:
            cur_data = df.xs(key=[mid, cond, day], level=[1, 2, 3])
            if cur_data.shape[0] > 0:
                traces = {idx: hv.Curve((np.arange(5400), np.transpose(row[1]))).opts(style={'alpha': 0.5})
                          for idx, row in enumerate(cur_data.itertuples())}
                means = hv.Curve((np.arange(5400), cur_data.loc[:, 'C_df'].mean(axis=0)), 
                                 group='Traces', label='Mean Trace').opts(style={'color': 'black'})
                ndoverlay = hv.NdOverlay(traces, kdims=['Neuron Number']).relabel(group='Traces', label='Raw_Traces', depth=1)
                overlay = ndoverlay * means
                holomap[mid] = overlay
            if mid in holomap.keys():
                gridspace[day, cond] = holomap
gridspace

In [ ]:
gb = df.groupby(level='conditionID', axis=0)
vals = {}
for name, group in gb:
    vals[name] = np.mean(group.values, axis=0)[0]
    
x_data = np.arange(vals['HYPO'].shape[0])
df_avg = pd.DataFrame(vals)
df_avg['time'] = x_data
df_avg.head()

In [ ]:
%%opts Curve [width=1200 height=400 tools=['hover']]

kdims = ['time']
vdims = ['C_df']
c = hv.Curve((df_avg['time'], df_avg['HYPER']), label='Average Hyper', group='All Mice', kdims=kdims, vdims=vdims) *\
    hv.Curve((df_avg['time'], df_avg['HYPO']), label='Average Hypo', group='All Mice', kdims=kdims, vdims=vdims)
c